- for more information -> https://data.calgary.ca/Environment/Solar-Energy-Production/ytdn-2qsp
- API docs -> https://dev.socrata.com/foundry/data.calgary.ca/ytdn-2qsp
- https://data.calgary.ca/resource/ytdn-2qsp.json

In [1]:
import pandas as pd 
import json

In [2]:
df = pd.read_csv("data/solar-energy-production.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281145 entries, 0 to 281144
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              281145 non-null  object 
 1   id                281145 non-null  int64  
 2   address           281145 non-null  object 
 3   date              281145 non-null  object 
 4   kWh               281145 non-null  float64
 5   public_url        281145 non-null  object 
 6   installationDate  281145 non-null  object 
 7   uid               281145 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 17.2+ MB


In [4]:
# list all the locations in the dataset
locations = df["name"].unique().tolist()
print(locations)

# a map of the locations can be found here -> https://data.calgary.ca/dataset/City-of-Calgary-Solar-Photovoltaic-Sites/vrdj-ycb5
df_sites = pd.read_csv("data/solar-sites.csv")
df_sites.head()

['Calgary Fire Hall Headquarters', 'Whitehorn Multi-Service Centre', 'Southland Leisure Centre', 'Hillhurst Sunnyside Community Association', 'Glenmore Water Treatment Plant', 'City of Calgary North Corporate Warehouse', 'Richmond - Knob Hill Community Hall', 'CFD Firehall #7', 'Manchester Building M', 'Telus Spark', 'Bearspaw Water Treatment Plant']


,id,name,address_orig,address,latitude,longitude,location,installationDate,public_url,manufacturerName,modelName,maximumPower
0,NaN,Fire Station No. 22 - Temple,7199 Temple DR NW,7199 TEMPLE DR NE,51.091770,-113.957493,"(51.09177006, -113.9574929)",2011/01/01,NaN,NaN,NaN,NaN
1,NaN,Fire Station No. 24 - Cedarbrae,2607 106 AVE SW,2607 106 AV SW,50.957431,-114.121390,"(50.95743078, -114.1213903)",2011/01/01,NaN,NaN,NaN,NaN
2,NaN,69th St. Park and Ride,7085 17 AVE SW,7085 17 AV SW,51.037227,-114.189256,"(51.03722709, -114.1892563)",2011/01/01,NaN,NaN,NaN,NaN
3,NaN,Fire Station No. 8 – Rosscarrock,1720 45 ST SW,1720 45 ST SW,51.038673,-114.152118,"(51.03867255, -114.1521183)",2011/01/01,NaN,NaN,NaN,NaN
4,NaN,Fire Station No. 5 – South Calgary,3129 14 ST SW,3129 14 ST SW,51.026009,-114.094948,"(51.0260092, -114.0949484)",2012/01/01,NaN,NaN,NaN,NaN


![Map of Calagry](calagry-sites.png "Map of Calagry")

In [5]:
df_fire_hall_hq = df.loc[df["name"] == "Calgary Fire Hall Headquarters"]
df_fire_hall_hq = df_fire_hall_hq.reset_index(drop=True)
df_fire_hall_hq = df_fire_hall_hq.sort_values(by="date")
df_fire_hall_hq.tail(5)

,name,id,address,date,kWh,public_url,installationDate,uid
26978,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023/08/19 08:00:00 PM,0.107,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-08-19 20:00:00
26967,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023/08/19 09:00:00 AM,4.105,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-08-19 09:00:00
26968,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023/08/19 10:00:00 AM,7.015,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-08-19 10:00:00
26969,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023/08/19 11:00:00 AM,7.429,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-08-19 11:00:00
26970,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023/08/19 12:00:00 PM,9.362,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-08-19 12:00:00


In [6]:
import plotly.express as px

# x and y given as DataFrame columns
fig = px.scatter(
      df_fire_hall_hq, 
      x="date", y="kWh", 
      opacity=0.8, 
      width=5000, height=400, 
      color_discrete_sequence=px.colors.qualitative.Dark24
)
fig.update_layout(
    plot_bgcolor="#e6e6e6",
    paper_bgcolor="#ffffff"
)
fig.show()


In [7]:
df["name"].unique()

array(['Calgary Fire Hall Headquarters', 'Whitehorn Multi-Service Centre',
       'Southland Leisure Centre',
       'Hillhurst Sunnyside Community Association',
       'Glenmore Water Treatment Plant',
       'City of Calgary North Corporate Warehouse',
       'Richmond - Knob Hill Community Hall', 'CFD Firehall #7',
       'Manchester Building M', 'Telus Spark',
       'Bearspaw Water Treatment Plant'], dtype=object)

In [8]:
min_date = min(df_fire_hall_hq["date"])
max_date = max(df_fire_hall_hq["date"])

print(min_date)
print(max_date)

2016/11/24 02:00:00 PM
2023/08/19 12:00:00 PM


## Loading weather data

In [9]:
# import requests

# fire_hall_hq_location = ("51.01618884", "-114.036667015348") # lat, long
# url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/51.01618884%2C%20-114.036667015348/2023-07-10/2023-08-20?unitGroup=metric&include=hours&key=GWNLWN96VNZPG2ECFGKSU66N8&contentType=json"
# response = requests.get(url)
# print(response.text)

In [10]:
# weather_fire_hall_hq_0710_0820 = response.json()
# print(json.dumps(weather_fire_hall_hq_0710_0820, indent=4))

In [11]:
# with open("data/weather-0710-0820.json", "w") as f:
#     json.dump(weather_fire_hall_hq_0710_0820, f)

In [12]:
# filter by date
df_fire_hall_hq_filtered = df_fire_hall_hq[df_fire_hall_hq["date"] >= "2023/07/10 01:00:00 PM"]

# Convert the "date" column to a datetime object
df_fire_hall_hq_filtered["date"] = pd.to_datetime(df_fire_hall_hq_filtered["date"], format="%Y/%m/%d %I:%M:%S %p")

# Sort the dataframe based on the datetime
df_fire_hall_hq_filtered = df_fire_hall_hq_filtered.sort_values(by="date")

# change date to perform filtering operations, rename for additional date column containing only the date
df_fire_hall_hq_filtered = df_fire_hall_hq_filtered.rename(columns={"date": "datetime"})
df_fire_hall_hq_filtered["datetime"] = pd.to_datetime(df_fire_hall_hq_filtered["datetime"])

# separate date and time
df_fire_hall_hq_filtered["date"] = df_fire_hall_hq_filtered["datetime"].dt.date
df_fire_hall_hq_filtered["time"] = df_fire_hall_hq_filtered["datetime"].dt.time

df_fire_hall_hq_filtered.head(3)

C:\Users\leopu\AppData\Local\Temp\ipykernel_8308\2424077081.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,name,id,address,datetime,kWh,public_url,installationDate,uid,date,time
26701,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023-07-10 05:00:00,0.003,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-07-10 05:00:00,2023-07-10,05:00:00
26702,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023-07-10 06:00:00,0.303,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-07-10 06:00:00,2023-07-10,06:00:00
26703,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023-07-10 07:00:00,1.144,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-07-10 07:00:00,2023-07-10,07:00:00


In [13]:
list_names = [
    "temp", "feelslike", "humidity", 
    "dew", "snow", "snowdepth", 
    "windgust", "windspeed", "winddir", 
    "pressure", "visibility", "cloudcover", 
    "solarradiation", "solarenergy", "uvindex", 
    "conditions", "stations"
]

list_collection = {name: [] for name in list_names}

In [14]:
date_list = df_fire_hall_hq_filtered["date"].tolist()
time_list = df_fire_hall_hq_filtered["time"].tolist()

# reverse the list 
date_list = list(reversed(date_list))
time_list = list(reversed(time_list))

In [15]:
with open("data/weather-0710-0820.json", "r") as f:
    weather_data = json.load(f)

enriched_dates = []
# first loop through all dates and times in the dataframe 
for d, t in zip(date_list, time_list):
    # only use the date, not the time (it's always 00:00:00)
    d = str(d)[:10]

    # then loop through all the days in the weather json
    for w in range(len(weather_data["days"])):
        w = int(w)

        #print(str(d)[:10])
        #print(weather_data["days"][w]["datetime"])
        
        # if dates are identical ...
        if weather_data["days"][w]["datetime"] == d:

            #print("Match!")
            enriched_dates.append(d)
            # ... loop through hours
            for h in range(len(weather_data["days"][int(w)]["hours"])):

                # if hours are identical, append weather info 
                if weather_data["days"][w]["hours"][h]["datetime"] == str(t):

                    # by looping through all the list names
                    for l in list_names:
                        list_collection[l].append(
                            weather_data["days"][w]["hours"][h][l]
                        )

In [16]:
for l in list_names:
    df_fire_hall_hq_filtered[l] = list_collection[l][::-1] # make sure to reverse the list, as the weather data is in a different order

In [17]:
df_fire_hall_hq_filtered.head(3)

,name,id,address,datetime,kWh,public_url,installationDate,uid,date,time,...,windspeed,winddir,pressure,visibility,cloudcover,solarradiation,solarenergy,uvindex,conditions,stations
26701,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023-07-10 05:00:00,0.003,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-07-10 05:00:00,2023-07-10,05:00:00,...,6.5,197.0,1012.0,16.0,27.4,0.0,0.0,0.0,Partially cloudy,"[71877099999, CPST, 71495099999, 71526099999, ..."
26702,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023-07-10 06:00:00,0.303,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-07-10 06:00:00,2023-07-10,06:00:00,...,3.5,327.0,1012.5,16.0,30.0,39.0,0.1,0.0,Partially cloudy,"[71877099999, CPST, 71495099999, 71526099999, ..."
26703,Calgary Fire Hall Headquarters,314106,1212 42 AV SE,2023-07-10 07:00:00,1.144,https://monitoringpublic.solaredge.com/solared...,2016/11/07,3141062023-07-10 07:00:00,2023-07-10,07:00:00,...,5.5,247.0,1012.3,16.0,88.0,35.0,0.1,0.0,Partially cloudy,"[71877099999, CPST, 71495099999, 71526099999, ..."


In [18]:
df_fire_hall_hq_filtered[["date", "time", "temp"]]

,date,time,temp
26701,2023-07-10,05:00:00,15.4
26702,2023-07-10,06:00:00,13.1
26703,2023-07-10,07:00:00,15.0
26704,2023-07-10,08:00:00,16.1
26705,2023-07-10,09:00:00,18.3
...,...,...,...
26974,2023-08-19,16:00:00,14.8
26975,2023-08-19,17:00:00,14.9
26976,2023-08-19,18:00:00,16.2
26977,2023-08-19,19:00:00,15.8
